In [1]:
import pandas as pd
import numpy as np
import requests

url = "https://cocl.us/Geospatial_data"
df_lat_long = pd.read_csv(url)

from bs4 import BeautifulSoup

address = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text


soup = BeautifulSoup(address,'lxml')

My_table = soup.find('table',{'class':'wikitable sortable'})

Postcode = []
Borough = []
Neighbourhood = []

for row in My_table.findAll('tr'):
    cells = row.findAll('td')
    if len(cells)==3: #Only extract table body not heading
        Postcode.append(cells[0].find(text=True))
        Borough.append(cells[1].find(text=True))
        Neighbourhood.append(cells[2].find(text=True))
       
df=pd.DataFrame(Postcode,columns=['Postal Code'])
df['Borough']= Borough
df['Neighbourhood']=Neighbourhood

df2 = df[df.Borough != 'Not assigned']

df3 = df2.replace('Not assigned',np.nan, regex = True)

df3['Neighbourhood'] = df3['Neighbourhood'].fillna(value=df['Borough'])

df4 = df3.groupby(['Postal Code'], as_index=False).agg(lambda x: ', '.join(set(x)))

df5 = df4.replace('\n','', regex=True)

df6 = pd.merge(df5, df_lat_long, on ='Postal Code')
                                                                                                                                                                                                                                                                                            

In [2]:
import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans
!conda install -c conda-forge folium=0.5.0
import folium as folium


Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following NEW packages will be INSTALLED:

    altair:  2.2.2-py35_1 conda-forge
    branca:  0.3.1-py_0   conda-forge
    folium:  0.5.0-py_0   conda-forge
    vincent: 0.4.4-py_1   conda-forge

altair-2.2.2-p 100% |################################| Time: 0:00:00  56.62 MB/s
branca-0.3.1-p 100% |################################| Time: 0:00:00  28.62 MB/s
vincent-0.4.4- 100% |################################| Time: 0:00:00  33.11 MB/s
folium-0.5.0-p 100% |################################| Time: 0:00:00  34.74 MB/s


In [3]:
#reducing the data to just boroughs which contain the word 'Toronto'

df_just_Toronto= df6[df6['Borough'].str.contains('Toronto')]
df_just_Toronto.head()


,Postal Code,Borough,Neighbourhood,Latitude,Longitude
37,M4E,East Toronto,The Beaches,43.676357,-79.293031
41,M4K,East Toronto,"Riverdale, The Danforth West",43.679557,-79.352188
42,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572
43,M4M,East Toronto,Studio District,43.659526,-79.340923
44,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


In [4]:
#creating a map showing the location of Boroughs with Toronoto in the name.

latitude = 43.69
longitude = -79.42

map_Toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for latitude, longitude, Borough, Neighbourhood in zip(df_just_Toronto['Latitude'], df_just_Toronto['Longitude'], df_just_Toronto['Borough'], df_just_Toronto['Neighbourhood']):
    label = '{}, {}'.format(Neighbourhood, Borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [latitude, longitude],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Toronto)  
    
map_Toronto

In [5]:
#getting the data on the neighbourhoods from 4square.

CLIENT_ID = 'A3WDT5M3W04G5D1QVOE3OELHQMT5MGBFMTQI0D32OLPRAN5P' # your Foursquare ID
CLIENT_SECRET = '1VLYYYNELUYJ1QZJ5ZP330YKLPRDSBTRZ0MSKOWGJQL4BJ0C' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

LIMIT = 100
radius = 500

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)


Toronto_venues = getNearbyVenues(names=df_just_Toronto['Neighbourhood'],latitudes=df_just_Toronto['Latitude'],longitudes=df_just_Toronto['Longitude'])

print(Toronto_venues.shape)
Toronto_venues.head()

The Beaches
Riverdale, The Danforth West
The Beaches West, India Bazaar
Studio District
Lawrence Park
Davisville North
North Toronto West
Davisville
Moore Park, Summerhill East
Forest Hill SE, South Hill, Summerhill West, Rathnelly, Deer Park
Rosedale
St. James Town, Cabbagetown
Church and Wellesley
Harbourfront, Regent Park
Ryerson, Garden District
St. James Town
Berczy Park
Central Bay Street
Richmond, King, Adelaide
Harbourfront East, Toronto Islands, Union Station
Design Exchange, Toronto Dominion Centre
Victoria Hotel, Commerce Court
Roselawn
Forest Hill West, Forest Hill North
North Midtown, The Annex, Yorkville
Harbord, University of Toronto
Kensington Market, Grange Park, Chinatown
South Niagara, CN Tower, Bathurst Quay, Railway Lands, King and Spadina, Harbourfront West, Island airport
Stn A PO Boxes 25 The Esplanade
Underground city, First Canadian Place
Christie
Dovercourt Village, Dufferin
Little Portugal, Trinity
Exhibition Place, Brockton, Parkdale Village
High Park, The 

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
1,The Beaches,43.676357,-79.293031,Starbucks,43.678798,-79.298045,Coffee Shop
2,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood
3,"Riverdale, The Danforth West",43.679557,-79.352188,Pantheon,43.677621,-79.351434,Greek Restaurant
4,"Riverdale, The Danforth West",43.679557,-79.352188,Dolce Gelato,43.677773,-79.351187,Ice Cream Shop


In [6]:
#checking the number of venues returned per Neighborhood
Toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,55,55,55,55,55,55
Business Reply Mail Processing Centre 969 Eastern,19,19,19,19,19,19
Central Bay Street,83,83,83,83,83,83
Christie,15,15,15,15,15,15
Church and Wellesley,86,86,86,86,86,86
Davisville,37,37,37,37,37,37
Davisville North,9,9,9,9,9,9
"Design Exchange, Toronto Dominion Centre",100,100,100,100,100,100
"Dovercourt Village, Dufferin",18,18,18,18,18,18


In [7]:
# one hot encoding
Toronto_onehot = pd.get_dummies(Toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Toronto_onehot['Neighbourhood'] = Toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [Toronto_onehot.columns[-1]] + list(Toronto_onehot.columns[:-1])
Toronto_onehot = Toronto_onehot[fixed_columns]

Toronto_onehot.head()

Toronto_grouped = Toronto_onehot.groupby('Neighbourhood').mean().reset_index()
Toronto_grouped.head()

,Neighbourhood,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,Berczy Park,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
1,Business Reply Mail Processing Centre 969 Eastern,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.052632
2,Central Bay Street,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.012048,...,0.0,0.0,0.012048,0.000000,0.012048,0.000000,0.012048,0.000000,0.0,0.012048
3,Christie,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
4,Church and Wellesley,0.011628,0.011628,0.0,0.0,0.0,0.0,0.0,0.0,0.011628,...,0.0,0.0,0.000000,0.011628,0.000000,0.011628,0.000000,0.011628,0.0,0.011628


In [8]:
#five most common venues for each Neighbourhood

num_top_venues = 5

for hood in Toronto_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = Toronto_grouped[Toronto_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
            venue  freq
0     Coffee Shop  0.07
1      Restaurant  0.05
2    Cocktail Bar  0.05
3  Farmers Market  0.04
4     Cheese Shop  0.04


----Business Reply Mail Processing Centre 969 Eastern----
                venue  freq
0  Light Rail Station  0.11
1          Restaurant  0.05
2          Smoke Shop  0.05
3         Pizza Place  0.05
4                 Spa  0.05


----Central Bay Street----
                venue  freq
0         Coffee Shop  0.16
1                Café  0.07
2  Italian Restaurant  0.05
3        Burger Joint  0.04
4                 Bar  0.04


----Christie----
               venue  freq
0      Grocery Store  0.20
1               Café  0.20
2               Park  0.13
3  Convenience Store  0.07
4          Nightclub  0.07


----Church and Wellesley----
                 venue  freq
0  Japanese Restaurant  0.07
1          Coffee Shop  0.06
2     Sushi Restaurant  0.06
3              Gay Bar  0.05
4           Restaurant  0.03


----Davisville----
    

In [46]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

        
# create a new dataframe
neighbourhoods_venues_sorted = pd.DataFrame(columns=columns)
neighbourhoods_venues_sorted['Neighbourhood'] = Toronto_grouped['Neighbourhood']

for ind in np.arange(Toronto_grouped.shape[0]):
    neighbourhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Toronto_grouped.iloc[ind, :], num_top_venues)

neighbourhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Restaurant,Cocktail Bar,Café,Beer Bar,Italian Restaurant,Bakery,Steakhouse,Seafood Restaurant,Cheese Shop
1,Business Reply Mail Processing Centre 969 Eastern,Light Rail Station,Yoga Studio,Recording Studio,Smoke Shop,Skate Park,Brewery,Burrito Place,Butcher,Restaurant,Comic Shop
2,Central Bay Street,Coffee Shop,Café,Italian Restaurant,Burger Joint,Bar,Chinese Restaurant,Thai Restaurant,Salad Place,Bubble Tea Shop,Spa
3,Christie,Café,Grocery Store,Park,Baby Store,Diner,Italian Restaurant,Nightclub,Restaurant,Coffee Shop,Convenience Store
4,Church and Wellesley,Japanese Restaurant,Sushi Restaurant,Coffee Shop,Gay Bar,Restaurant,Burger Joint,Café,Pub,Men's Store,Mediterranean Restaurant


In [79]:
# set number of clusters
kclusters = 5

Toronto_grouped_clustering = Toronto_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:20] 

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 4, 0, 3], dtype=int32)

In [80]:
del neighbourhoods_venues_sorted['Cluster labels']

neighbourhoods_venues_sorted.insert(0,'Cluster labels', kmeans.labels_)

#list(neighbourhoods_venues_sorted)

Toronto_merged = df_just_Toronto

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
Toronto_merged = Toronto_merged.join(neighbourhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

Toronto_merged.head(10) # check the last columns!

,Postal Code,Borough,Neighbourhood,Latitude,Longitude,Cluster labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
37,M4E,East Toronto,The Beaches,43.676357,-79.293031,2,Coffee Shop,Pub,Neighborhood,Yoga Studio,Discount Store,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant
41,M4K,East Toronto,"Riverdale, The Danforth West",43.679557,-79.352188,0,Greek Restaurant,Coffee Shop,Ice Cream Shop,Bookstore,Italian Restaurant,Cosmetics Shop,Brewery,Bubble Tea Shop,Restaurant,Caribbean Restaurant
42,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572,0,Fast Food Restaurant,Ice Cream Shop,Board Shop,Liquor Store,Burger Joint,Fish & Chips Shop,Sandwich Place,Burrito Place,Steakhouse,Sushi Restaurant
43,M4M,East Toronto,Studio District,43.659526,-79.340923,0,Café,Coffee Shop,Italian Restaurant,Bakery,American Restaurant,Yoga Studio,Coworking Space,Seafood Restaurant,Sandwich Place,Cheese Shop
44,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,4,Park,Swim School,Dim Sum Restaurant,Bus Line,Yoga Studio,Dog Run,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space
45,M4P,Central Toronto,Davisville North,43.712751,-79.390197,0,Dance Studio,Breakfast Spot,Burger Joint,Food & Drink Shop,Clothing Store,Hotel,Sandwich Place,Park,Gym,Event Space
46,M4R,Central Toronto,North Toronto West,43.715383,-79.405678,0,Clothing Store,Sporting Goods Shop,Coffee Shop,Yoga Studio,Chinese Restaurant,Dessert Shop,Rental Car Location,Diner,Salon / Barbershop,Sandwich Place
47,M4S,Central Toronto,Davisville,43.704324,-79.388790,0,Dessert Shop,Sandwich Place,Pizza Place,Seafood Restaurant,Restaurant,Café,Coffee Shop,Sushi Restaurant,Italian Restaurant,Farmers Market
48,M4T,Central Toronto,"Moore Park, Summerhill East",43.689574,-79.383160,3,Playground,Park,Diner,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant
49,M4V,Central Toronto,"Forest Hill SE, South Hill, Summerhill West, R...",43.686412,-79.400049,0,Coffee Shop,Pub,Pizza Place,American Restaurant,Convenience Store,Sports Bar,Bagel Shop,Supermarket,Sushi Restaurant,Fried Chicken Joint


In [81]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Toronto_merged['Latitude'], Toronto_merged['Longitude'], Toronto_merged['Neighbourhood'], Toronto_merged['Cluster labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [92]:
Toronto_merged.loc[Toronto_merged['Cluster labels'] == 0, Toronto_merged.columns[[2] + list(range(5, Toronto_merged.shape[1]))]]

,Neighbourhood,Cluster labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
41,"Riverdale, The Danforth West",0,Greek Restaurant,Coffee Shop,Ice Cream Shop,Bookstore,Italian Restaurant,Cosmetics Shop,Brewery,Bubble Tea Shop,Restaurant,Caribbean Restaurant
42,"The Beaches West, India Bazaar",0,Fast Food Restaurant,Ice Cream Shop,Board Shop,Liquor Store,Burger Joint,Fish & Chips Shop,Sandwich Place,Burrito Place,Steakhouse,Sushi Restaurant
43,Studio District,0,Café,Coffee Shop,Italian Restaurant,Bakery,American Restaurant,Yoga Studio,Coworking Space,Seafood Restaurant,Sandwich Place,Cheese Shop
45,Davisville North,0,Dance Studio,Breakfast Spot,Burger Joint,Food & Drink Shop,Clothing Store,Hotel,Sandwich Place,Park,Gym,Event Space
46,North Toronto West,0,Clothing Store,Sporting Goods Shop,Coffee Shop,Yoga Studio,Chinese Restaurant,Dessert Shop,Rental Car Location,Diner,Salon / Barbershop,Sandwich Place
47,Davisville,0,Dessert Shop,Sandwich Place,Pizza Place,Seafood Restaurant,Restaurant,Café,Coffee Shop,Sushi Restaurant,Italian Restaurant,Farmers Market
49,"Forest Hill SE, South Hill, Summerhill West, R...",0,Coffee Shop,Pub,Pizza Place,American Restaurant,Convenience Store,Sports Bar,Bagel Shop,Supermarket,Sushi Restaurant,Fried Chicken Joint
51,"St. James Town, Cabbagetown",0,Coffee Shop,Restaurant,Park,Italian Restaurant,Bakery,Café,Pub,Pizza Place,Pharmacy,Snack Place
52,Church and Wellesley,0,Japanese Restaurant,Sushi Restaurant,Coffee Shop,Gay Bar,Restaurant,Burger Joint,Café,Pub,Men's Store,Mediterranean Restaurant
53,"Harbourfront, Regent Park",0,Coffee Shop,Bakery,Café,Pub,Park,Theater,Mexican Restaurant,Breakfast Spot,Restaurant,Electronics Store


In [88]:
Toronto_merged.loc[Toronto_merged['Cluster labels'] == 1, Toronto_merged.columns[[2] + list(range(5, Toronto_merged.shape[1]))]]

,Neighbourhood,Cluster labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
63,Roselawn,1,Garden,Yoga Studio,Dog Run,Filipino Restaurant,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store


In [89]:
Toronto_merged.loc[Toronto_merged['Cluster labels'] == 2, Toronto_merged.columns[[2] + list(range(5, Toronto_merged.shape[1]))]]

,Neighbourhood,Cluster labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
37,The Beaches,2,Coffee Shop,Pub,Neighborhood,Yoga Studio,Discount Store,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant


In [90]:
Toronto_merged.loc[Toronto_merged['Cluster labels'] == 3, Toronto_merged.columns[[2] + list(range(5, Toronto_merged.shape[1]))]]

,Neighbourhood,Cluster labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
48,"Moore Park, Summerhill East",3,Playground,Park,Diner,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant
50,Rosedale,3,Park,Playground,Trail,Yoga Studio,Discount Store,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant
64,"Forest Hill West, Forest Hill North",3,Park,Trail,Sushi Restaurant,Jewelry Store,Yoga Studio,Dog Run,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space


In [91]:
Toronto_merged.loc[Toronto_merged['Cluster labels'] == 4, Toronto_merged.columns[[2] + list(range(5, Toronto_merged.shape[1]))]]

,Neighbourhood,Cluster labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
44,Lawrence Park,4,Park,Swim School,Dim Sum Restaurant,Bus Line,Yoga Studio,Dog Run,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space


Cluster 0 is rich in eating and drinking locations
Cluster 1 contains the gardens
Cluster 2 contains the neighbourhoods
Cluster 3 contains the park, playground and trails
Cluster 4 containes the swim school and park